In [101]:
# packages 
import numpy as np
import pandas as pd
import math
from sklearn import linear_model
from sklearn import tree
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import model_selection
from sklearn.model_selection import train_test_split

In [102]:
# load data
path = "~/../../../../project/biocomplexity/sdad/projects_data/mc/data_commons/dc_education_training" # specify the path to MEPS and ACS files here
meps = pd.read_excel(path+"h217.xlsx")

In [103]:
meps.head
# DUPERSID - PERSON ID (DUID + PID), PID - person's number
cols_select = ["DUPERSID", "PID", 
              "ADEXPL4", "ADEZUN4", "ADFHLP4", 
              "RACETHX","RACEV1X", "RACEV2X", 
              "BORNUSA", "OTHLGSPK", "WHTLGSPK" ,"YRSINUS", 
              "EDUCYR", "HIDEG", 
              "SEX",
              "AGEY2X",
              "MARRYY2X",
              "POVCATY1", "POVCATY2", "POVLEVY1", "POVLEVY2"]
questions = meps[cols_select]

In [104]:
# encode questions
# drop -1 INAPPLICABLE and -15 CANNOT BE COMPUTED
questions = questions.drop(questions[questions.ADEXPL4 < 0].index)
questions = questions.drop(questions[questions.ADEZUN4 < 0].index)
questions = questions.drop(questions[questions.ADFHLP4 < 0].index)
# total score -- add scores together
questions["tot_score"] = questions.ADEXPL4 + questions.ADEZUN4 + questions.ADFHLP4
#questions.tot_score.describe()
# de-mean tot_score
questions["std_score"] = questions.tot_score - questions.tot_score.mean() 
#questions.std_score.describe()
# helath literacy above average if the std_score is positive, and below average if negative
questions["above_av"] = 1
questions.loc[questions['std_score'] <0 , 'above_av'] = 0 # above average means that a person USUALLY health literate
#questions.above_av.describe()

In [131]:
# encode chars as in ACS 

# age groups
questions["age_less_18"] = 0
questions.loc[questions['AGEY2X'] < 18, 'age_less_18'] = 1
questions["age_18_24"] = 0
questions.loc[(questions['AGEY2X'] <= 24) & (questions['AGEY2X'] >= 18), 'age_18_24'] = 1
questions["age_25_39"] = 0
questions.loc[(questions['AGEY2X'] <= 39) & (questions['AGEY2X'] >= 25), 'age_25_39'] = 1
questions["age_40_49"] = 0
questions.loc[(questions['AGEY2X'] <= 49) & (questions['AGEY2X'] >= 40), 'age_40_49'] = 1
questions["age_50_64"] = 0
questions.loc[(questions['AGEY2X'] <= 64) & (questions['AGEY2X'] >= 50), 'age_50_64'] = 1
questions["age_65_74"] = 0
questions.loc[(questions['AGEY2X'] <= 74) & (questions['AGEY2X'] >= 65), 'age_65_74'] = 1
# reference category
#questions["age_above_74"] = 0
#questions.loc[questions['AGEY2X'] > 74, 'age_above_74'] = 1

# gender
# reference category - male
questions["female"] = 0
questions.loc[questions['SEX'] == 2, 'female'] = 1

# race/ethnicity 
# reference category - white 
# questions["white"] = 0
# questions.loc[questions['RACETHX'] == 2, 'white'] = 1
questions["black"] = 0
questions.loc[questions['RACETHX'] == 3, 'black'] = 1
questions["hispanic"] = 0
questions.loc[questions['RACETHX'] == 1, 'hispanic'] = 1
questions["asian_pi"] = 0
questions.loc[(questions['RACEV1X'] == 4) & (questions['RACETHX'] != 1), 'asian_pi'] = 1
questions["native"] = 0
questions.loc[(questions['RACEV1X'] == 3) & (questions['RACETHX'] != 1), 'native'] = 1
questions["mixed"] = 0
questions.loc[questions['RACETHX'] == 5, 'mixed'] = 1

# education
# reference category - no education data
#questions['edu_mis'] = 0
#questions.loc[(questions['HIDEG'] == -7) | (questions['HIDEG'] == -8) | (questions['HIDEG'] == -15), 'edu_mis'] = 1
questions['HS_GED'] = 0
questions.loc[(questions['HIDEG'] == 2) | (questions['HIDEG'] == 3), 'HS_GED'] = 1
questions['no_edu'] = 0
questions.loc[questions['HIDEG'] == 1, 'no_edu'] = 1
questions['BS_degree'] = 0
questions.loc[questions['HIDEG'] == 4, 'BS_degree'] = 1
questions['above_BS'] = 0
questions.loc[(questions['HIDEG'] == 5) | (questions['HIDEG'] == 6), 'above_BS'] = 1
questions['oth_degree'] = 0
questions.loc[questions['HIDEG'] == 7, 'oth_degree'] = 1

# income 
# reference - above poverty line
questions['below_PVL'] = 0
questions.loc[questions['POVCATY2'] == 1, 'below_PVL'] = 1

# marital status
# reference - married
#questions["married"] = 0
#questions.loc[questions['MARRYY2X'] == 1, 'married'] = 1
questions["wid_sep_div"] = 0
questions.loc[(questions['MARRYY2X'] == 2) | (questions['MARRYY2X'] == 3)
              | (questions['MARRYY2X'] == 4), 'wid_sep_div'] = 1
questions["nev_married"] = 0
questions.loc[questions['MARRYY2X'] == 5, 'nev_married'] = 1

# language spoken at home
# reference - no info
#questions['lang_mis'] = 0 
#questions.loc[(questions['OTHLGSPK'] == -15) | (questions['OTHLGSPK'] == -8), 'lang_mis'] = 1
questions['lang_eng'] = 0
questions.loc[questions['OTHLGSPK'] == 2, 'lang_eng'] = 1
questions['lang_oth'] = 0
questions.loc[questions['OTHLGSPK'] == 1, 'lang_oth'] = 1

# born in the US
# reference - not born in the US
questions['not_born_us'] = 0
questions.loc[questions['BORNUSA'] == 1, 'not_born_us'] = 2
questions['born_us'] = 0
questions.loc[questions['BORNUSA'] == 1, 'born_us'] = 1
#questions['born_mis'] = 0
#questions.loc[(questions['BORNUSA'] == -15) | (questions['BORNUSA'] == -7) 
# | (questions['BORNUSA'] == -8) , 'born_mis'] = 1


In [132]:
questions.head
#questions.AGEY2X.unique()
#questions.AGEY2X.describe()
#questions.RACETHX.unique()
#questions.SEX.unique()
#questions.SEX.describe()
#questions.HIDEG.unique()
#questions.OTHLGSPK.unique()
#questions.BORNUSA.unique()

<bound method NDFrame.head of          DUPERSID  PID  ADEXPL4  ADEZUN4  ADFHLP4  RACETHX  RACEV1X  RACEV2X  \
8      2320008102  102        4        4        4        4        4        4   
12     2320018102  102        4        4        1        2        1        1   
13     2320018103  103        1        4        2        2        1        1   
15     2320019102  102        3        2        2        2        1        1   
18     2320022101  101        4        4        1        2        1        1   
...           ...  ...      ...      ...      ...      ...      ...      ...   
14050  2329677102  102        4        4        2        2        1        1   
14056  2329681102  102        3        4        2        2        1        1   
14057  2329682101  101        4        4        1        2        1        1   
14058  2329682102  102        3        3        2        2        1        1   
14062  2329684102  102        3        4        2        2        1        1   

       BO

In [134]:
y_col = ["std_score"]
#y_col = ["above_av"]
X_cols = ["age_less_18", "age_18_24", "age_25_39", "age_40_49", "age_50_64", "age_65_74", 
             "female", 
             "black", "hispanic", "asian_pi", "native", "mixed", 
             "HS_GED", "no_edu", "BS_degree", "above_BS", "oth_degree",
             "below_PVL", 
             "wid_sep_div", "nev_married", 
             "lang_eng", "lang_oth",
             "born_us", "not_born_us"]
y = questions[y_col]
X = questions[X_cols]

In [135]:
# split the data into train and test set
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.25, random_state=42)
y_test

,std_score
14021,-1.994982
13607,3.005018
10457,0.005018
5193,0.005018
1777,0.005018
...,...
898,-2.994982
2211,1.005018
13366,-2.994982
11937,3.005018


In [111]:
# linear regression model
lin_reg = linear_model.LinearRegression()
lin_reg.fit(np.array(X_train), y_train)
y_pred_lr = lin_reg.predict(X_test)
mae=metrics.mean_absolute_error(y_test, y_pred_lr)
mse=metrics.mean_squared_error(y_test, y_pred_lr)
# Printing the metrics
print('R2 square:',metrics.r2_score(y_test, y_pred_lr))
print('MAE: ', mae)
print('MSE: ', mse)

R2 square: -0.004109694920850782
MAE:  1.240409215018545
MSE:  2.707319545118017


In [136]:
# decision tree regressor
tree_reg = tree.DecisionTreeRegressor()
tree_reg.fit(np.array(X_train), y_train)
y_pred_tr = tree_reg.predict(X_test)
mae=metrics.mean_absolute_error(y_test, y_pred_tr)
mse=metrics.mean_squared_error(y_test, y_pred_tr)
# Printing the metrics
print('Decision Tree: ', tree_reg.score(X_test,y_test))
print('R2 square:',metrics.r2_score(y_test, y_pred_tr))
print('MAE: ', mae)
print('MSE: ', mse)

Decision Tree:  -0.4281521253730547
R2 square: -0.4281521253730547
MAE:  1.4930016822635075
MSE:  3.8506392100208555


In [113]:
# random forest:
rf_reg = RandomForestRegressor(n_estimators=100)
rf_reg.fit(np.array(X_train), y_train.values.ravel())
y_pred_rf = rf_reg.predict(X_test)
mae=metrics.mean_absolute_error(y_test, y_pred_rf)
mse=metrics.mean_squared_error(y_test, y_pred_rf)
# Printing the metrics
print('Random Forest: ', rf_reg.score(X_test,y_test))
print('R2 square:',metrics.r2_score(y_test, y_pred_rf))
print('MAE: ', mae)
print('MSE: ', mse)

Random Forest:  -0.14913123101146475
R2 square: -0.14913123101146475
MAE:  1.3443021145353728
MSE:  3.098332241347491


In [114]:
# Logistic Regression (only for binary outcome: above/below average)
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression(random_state=0)
log_reg.fit(X_train, y_train.values.ravel())
y_pred_lg = log_reg.predict(X_test)
mae=metrics.mean_absolute_error(y_test, y_pred_lg)
mse=metrics.mean_squared_error(y_test, y_pred_lg)
# Printing the metrics
print('Logistic Regression Score: ', log_reg.score(X_test,y_test))
print('R2 square:',metrics.r2_score(y_test, y_pred_lg))
print('MAE: ', mae)
print('MSE: ', mse)

ValueError: Unknown label type: 'continuous'

In [115]:
from sklearn.ensemble import GradientBoostingRegressor
gb_reg = GradientBoostingRegressor(random_state=0)
gb_reg.fit(X_train, y_train.values.ravel())
y_pred_gb = gb_reg.predict(X_test)
mae=metrics.mean_absolute_error(y_test, y_pred_gb)
mse=metrics.mean_squared_error(y_test, y_pred_gb)
# Printing the metrics
print('Gradient Boosting Regressor Score: ', gb_reg.score(X_test,y_test))
print('R2 square:',metrics.r2_score(y_test, y_pred_gb))
print('MAE: ', mae)
print('MSE: ', mse)

Gradient Boosting Regressor Score:  -0.040067818302294445
R2 square: -0.040067818302294445
MAE:  1.2647076257435148
MSE:  2.8042712334930813


In [116]:
from sklearn.metrics import mean_squared_error
print(mean_squared_error(y_test, y_pred_lr))
print(mean_squared_error(y_test, y_pred_tr))
print(mean_squared_error(y_test, y_pred_rf))

2.707319545118017
3.8616047091681733
3.098332241347491


In [169]:
# upload ACS data
acs_tr = pd.read_csv(path+"HL_X_pred_tr.csv", index_col="GEOID")
acs_ct = pd.read_csv(path+"HL_X_pred_ct.csv", index_col="GEOID")
acs_hd = pd.read_csv(path+"HL_X_pred_hd.csv", index_col="GEOID")

In [163]:
# predict using the decision tree
acs_tr = np.nan_to_num(acs_tr)
acs_ct = np.nan_to_num(acs_ct)
acs_hd = np.nan_to_num(acs_hd)

y_acs_tr = tree_reg.predict(acs_tr)
y_acs_ct = tree_reg.predict(acs_ct)
y_acs_hd = tree_reg.predict(acs_hd)
type(y_acs_hd.astype(int))

numpy.ndarray

In [173]:
hl_est_tr = pd.DataFrame(y_acs_tr.tolist())
hl_est_tr["GEOID"] = acs_tr.index

hl_est_ct = pd.DataFrame(y_acs_ct.tolist())
hl_est_ct["GEOID"] = acs_ct.index

hl_est_hd = pd.DataFrame(y_acs_hd.tolist())
hl_est_hd["GEOID"] = acs_hd.index

In [174]:
hl_est_tr.to_csv('hl_est_tr.csv',
                  index=False)
hl_est_ct.to_csv('hl_est_ct.csv',
                  index=False)
hl_est_hd.to_csv('hl_est_hd.csv',
                  index=False)